In [33]:
from docx import Document
from docx.shared import Pt
import pandas as pd
from datetime import datetime
import os
import re
import locale

In [34]:
excel_path = "Rektor Cup 2024.xlsx"
data = pd.read_excel(excel_path, sheet_name = "Individu")

In [35]:
template_path = "template.docx"

In [36]:
indv_output_folder = "output_letters/rektorcup/indv"
team_output_folder = "output_letters/rektorcup/team"
if not os.path.exists(indv_output_folder):
    os.makedirs(indv_output_folder)
    
if not os.path.exists(team_output_folder):
    os.makedirs(team_output_folder)

In [37]:
def format_dates(row):
    locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
    
    start_date = pd.to_datetime(row['Start Date'])
    end_date = pd.to_datetime(row['End Date'])
    
    # Format day and date (e.g., "Selasa - Kamis/26 - 28 April 2024")
    start_day = start_date.strftime("%A")  # Day of the week
    start_formatted = start_date.strftime("%d %B %Y")  # Date in "26 April 2024" format
    end_day = end_date.strftime("%A")
    end_formatted = end_date.strftime("%d %B %Y")
    
    return f"{start_day} - {end_day}/{start_formatted} - {end_formatted}"

In [38]:
def format_signatures(row):
    locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
    start_date = pd.to_datetime(row['Start Date'])
    
    # Format day and date (e.g., "Selasa - Kamis/26 - 28 April 2024")
    start_formatted = start_date.strftime("%d %B %Y")  # Date in "26 April 2024" format
    
    return f"Surabaya, {start_formatted}"

In [39]:
def replace_surabaya_paragraph(doc, row):
    """Replace 'Surabaya, <date>' in paragraphs and output changes."""
    signature_date = format_signatures(row)  # Format the new date
    pattern = r"Surabaya,\s+\d{1,2}\s+\w+\s+\d{4}"  # Regex pattern for 'Surabaya, <date>'
    
    for paragraph in doc.paragraphs:
        if re.search(pattern, paragraph.text):
            old_text = paragraph.text  # Save the original text for logging
            paragraph.text = re.sub(pattern, signature_date, paragraph.text)  # Replace text
            print(f"Changed: '{old_text}' to '{paragraph.text}'")  # Output the change

In [40]:
def sanitize_filename(filename):
    return "".join(c for c in filename if c.isalnum() or c in " _-()").strip()

In [41]:
doc = Document(template_path)

In [42]:
print("\nTables:")
for table_index, table in enumerate(doc.tables):
    print(f"Table {table_index + 1}:")
    for row_index, row in enumerate(table.rows):
        row_data = [repr(cell.text) for cell in row.cells]
        print(f"  Row {row_index + 1}: {row_data}")


Tables:
Table 1:
  Row 1: ["''", "''"]
Table 2:
  Row 1: ["'No'", "'Nama'", "'NIM/NIK'", "'Departemen/Prodi'"]
  Row 2: ["'1'", "'Rean Mitasari'", "'0731108905'", "'Dosen Pendamping'"]
  Row 3: ["'2'", "'Jocelyn Vanessa Meidy Winarto'", "'0406012110026'", "'HTB'"]
Table 3:
  Row 1: ["'Tugas'", "':'", "'Mengikuti Lomba CAK NING UC 2024'"]
  Row 2: ["'Hari/Tanggal'", "':     '", "'Rabu - Kamis/27 Maret - 18 April 2024'"]
  Row 3: ["'Tempat\\n\\n'", "':\\n\\n'", "'Universitas Ciputra Surabaya'"]


In [43]:
for index, row in data.iterrows():
    table = doc.tables[1]
    
    for _ in range(len(table.rows) - 1):
        table._tbl.remove(table.rows[1]._tr)
    
    lecturer_name = row['Nama Dosen PA']
    lecturer_nidn = row['NIDN Dosen PA']

    # Add lecturer information
    lecturer_row = table.add_row().cells
    lecturer_row[0].text = "1"  # No
    lecturer_row[1].text = lecturer_name  # Nama
    lecturer_row[2].text = str(lecturer_nidn)  # NIM/NIK
    lecturer_row[3].text = "Dosen Pendamping"  # Departemen/Prodi
    
    student_name = row['Name']
    student_nis = row['NIS']
    student_department = row['Major']
        
    # Add student information
    student_row = table.add_row().cells
    student_row[0].text = "2"  # N
    student_row[1].text = student_name  # Nama
    student_row[2].text = str(student_nis)  # NIM/NIK
    student_row[3].text = student_department  # Departemen/Prodi
    
    info_table = doc.tables[2]

    if len(info_table.rows) > 0:
        tugas_cell = info_table.rows[0].cells[2]
        tugas_cell.text = f"Mengikuti {row['Activity']}"

    if len(info_table.rows) > 1:
        haritanggal_cell = info_table.rows[1].cells[2]
        haritanggal_cell.text = format_dates(row)
    
    replace_surabaya_paragraph(doc, row)
    
    for paragraph in doc.paragraphs:
        if "No. :" in paragraph.text:  # Match paragraphs with "No. :"
            old_text = paragraph.text
            # Use a regex to replace the entire identifier after "No. :"
            new_text = re.sub(r"No\. :\s*[\w/-]+", f"No. : {row['Nomor ST']}", old_text)
            paragraph.text = new_text
            print(f"Updated 'Nomor ST' from: '{old_text}' to: '{paragraph.text}'")


    # Save the updated document
    sanitized_name = sanitize_filename(row['Name'])
    sanitized_activity = sanitize_filename(row['Activity'])
    indv_output_path = f"{indv_output_folder}/ST_{sanitized_name} ({sanitized_activity}).docx"
    
    print(f"Saving Word document to: {indv_output_path}")  
    
    doc.save(indv_output_path)
    print(f"Word document saved to {indv_output_path}")

Changed: 'Surabaya, 27 Maret 2024' to 'Surabaya, 04 April 2024'
Updated 'Nomor ST' from: 'No. : 00023H/UC-BMA/ST/III/2024
' to: 'No. : 00011U/UC-BMA/ST/III/2024
'
Saving Word document to: output_letters/rektorcup/indv/ST_Je Ivan Dhamma Pratama (Rector Cup Cabang Tenis Meja).docx
Word document saved to output_letters/rektorcup/indv/ST_Je Ivan Dhamma Pratama (Rector Cup Cabang Tenis Meja).docx
Changed: 'Surabaya, 04 April 2024' to 'Surabaya, 04 April 2024'
Updated 'Nomor ST' from: 'No. : 00011U/UC-BMA/ST/III/2024
' to: 'No. : 00011U/UC-BMA/ST/III/2024
'
Saving Word document to: output_letters/rektorcup/indv/ST_Michael Alessandro (Rector Cup Cabang Tenis Meja).docx
Word document saved to output_letters/rektorcup/indv/ST_Michael Alessandro (Rector Cup Cabang Tenis Meja).docx
Changed: 'Surabaya, 04 April 2024' to 'Surabaya, 04 April 2024'
Updated 'Nomor ST' from: 'No. : 00011U/UC-BMA/ST/III/2024
' to: 'No. : 00011U/UC-BMA/ST/III/2024
'
Saving Word document to: output_letters/rektorcup/indv/

In [44]:
team_data = pd.read_excel(excel_path, sheet_name = "Team")

In [45]:
grouped_teams = team_data.groupby(['Code', 'Activity'])

In [ ]:
for (code, activity), team_group in grouped_teams:
    # Clear and prepare the table for team members
    table = doc.tables[1]
    
    # Remove existing rows (excluding header row)
    for _ in range(len(table.rows) - 1):
        table._tbl.remove(table.rows[1]._tr)
    
    # Add lecturer information
    lecturer_name = team_group.iloc[0]['Nama Dosen PA']
    lecturer_nidn = team_group.iloc[0]['NIDN Dosen PA']
    lecturer_row = table.add_row().cells
    lecturer_row[0].text = "1"  # No
    lecturer_row[1].text = lecturer_name  # Nama
    lecturer_row[2].text = str(lecturer_nidn)  # NIM/NIK
    lecturer_row[3].text = "Dosen Pendamping"  # Departemen/Prodi

    for idx, member in enumerate(team_group.itertuples(), start=2):
        student_row = table.add_row().cells
        student_row[0].text = str(idx)
        student_row[1].text = member.Name
        student_row[2].text = str(member.NIS)
        student_row[3].text = member.Major 
    
    info_table = doc.tables[2]

    if len(info_table.rows) > 0:
        tugas_cell = info_table.rows[0].cells[2]
        tugas_cell.text = f"Mengikuti {activity}"

    if len(info_table.rows) > 1:
        haritanggal_cell = info_table.rows[1].cells[2]
        haritanggal_cell.text = format_dates(team_group.iloc[0])
        
    replace_surabaya_paragraph(doc, team_group.iloc[0])
    
    for paragraph in doc.paragraphs:
        if "No. :" in ''.join(run.text for run in paragraph.runs):
            full_text = ''.join(run.text for run in paragraph.runs).strip()
            print(f"Original Full Text: '{full_text}'")
        
            nomor_st_value = team_group.iloc[0]['Nomor ST']
            print(f"Processing Nomor ST: {nomor_st_value}")

            new_text = re.sub(r"No\. :\s*[\w/-]+", f"No. : {nomor_st_value}", full_text)
            
            for run in paragraph.runs:
                run.text = '' 
            paragraph.runs[0].text = new_text

            paragraph.paragraph_format.space_after = Pt(12)
            
            print(f"Updated 'Nomor ST' to: '{new_text}'")

    # Save the updated document
    first_member_name = team_group.iloc[0]['Name']
    sanitized_name = sanitize_filename(first_member_name)
    sanitized_activity = sanitize_filename(activity)
    team_output_path = f"{team_output_folder}/ST_Team_{sanitized_name} ({sanitized_activity}).docx"
    
    print(f"Saving Word document to: {team_output_path}")  
    
    doc.save(team_output_path)
    print(f"Word document saved to {team_output_path}")


Changed: 'Surabaya, 04 April 2024' to 'Surabaya, 04 April 2024'
Original Full Text: 'No. : 00011U/UC-BMA/ST/III/2024'
Processing Nomor ST: 00011U/UC-BMA/ST/III/2024
Updated 'Nomor ST' to: 'No. : 00011U/UC-BMA/ST/III/2024'
Saving Word document to: output_letters/rektorcup/team/ST_Team_Daniel Richard (Rektor Cup Cabang Badminton).docx
Word document saved to output_letters/rektorcup/team/ST_Team_Daniel Richard (Rektor Cup Cabang Badminton).docx
Changed: 'Surabaya, 04 April 2024' to 'Surabaya, 04 April 2024'
Original Full Text: 'No. : 00011U/UC-BMA/ST/III/2024'
Processing Nomor ST: 00011U/UC-BMA/ST/III/2024
Updated 'Nomor ST' to: 'No. : 00011U/UC-BMA/ST/III/2024'
Saving Word document to: output_letters/rektorcup/team/ST_Team_Matthew Adirata (Rektor Cup Cabang Badminton).docx
Word document saved to output_letters/rektorcup/team/ST_Team_Matthew Adirata (Rektor Cup Cabang Badminton).docx
Changed: 'Surabaya, 04 April 2024' to 'Surabaya, 04 April 2024'
Original Full Text: 'No. : 00011U/UC-BMA/S

In [47]:
for (code, activity), team_group in grouped_teams:
    print(f"\nGrouping for Activity: {activity}")
    print("Team Members:")
    print(team_group[['Name']])  # Display relevant columns


Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
                    Name
0         Daniel Richard
1  Aurely Darline Markho

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
                     Name
2         Matthew Adirata
3  Delicia Earlin Istanto

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
                    Name
4  Brad Nicklaus Chandra
5       Elvira Alexandra

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
                   Name
6  Steve Marchello Jauw
7       Eugenia Shannon

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
               Name
8    Jeven Leonardo
9  Claudia Theresia

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
             Name
10  Jevon Gunawan
11      Ray Natha

Grouping for Activity: Rektor Cup Cabang Badminton
Team Members:
                     Name
12  Timothy Alvaro Lolang
13            Phylea Holy

Grouping for Activity: Rektor Cup Cabang Badm